## **Парсер музыки из Вконтакте в Яндекс.музыку.**##


Такой парсер это конечно же не то, чтобы серьезный проект. Просто однажды я хотел перенести большое количество моей музыки на новую платформу. А так как композиций было 1000+, то вручную это было бы сделать нереально. Тогда я решил все это запарсить. Примерно день ушел на разбор в VK_API, библотеки для yandex_music и самого написания этого кода.

Подгружаем и инициализируем все нужные библотеки.

In [ ]:
!pip install vk_api

In [ ]:
!pip install yandex_music

In [ ]:
!pip install keyboard

In [ ]:
import vk_api
from vk_api import audio
import requests
import os
from yandex_music import Client
import keyboard
import tqdm

Для начала нужно было выпарсить список музыки из вк. Следующий код для этого и нужен.

In [ ]:
REQUEST_STATUS_CODE = 200
login = input('Введите логин:')  # Номер телефона

In [ ]:
password = input('Введите пароль:')  # Пароль

In [ ]:
my_id = input('Введите id вк:')  # Ваш id vk

In [ ]:
name_dir = input(str('Введите название папки:'))

In [ ]:
path = str(input('Введите путь к к месту, где хотите создать папку:')) + '//' + name_dir

In [ ]:
mode = input('Достпуные режимы для парсинга:\n'
             '1: Названия всех доступных аудиозаписей заносятся в текстовый документ\n'
             '2: Все доступные аудиозаписи скачиваются.\n'
             'Названия тех, которые нельзя скачать заносятся в текстовый документ.\n'
             '3: Объединение первых двух.\n'
             'Названия заносятся в текстовый документ. Все достпуные скачиваются.\n'
             'Названия тех, что невозможно скачать, добавляются в отдельный текстовый документ\n'
             'Введите желаемый режим:')

In [ ]:
vk_session = vk_api.VkApi(login=login, password=password)
vk_session.auth()
vk = vk_session.get_api()  # Теперь можно обращаться к методам API как к обычным классам
vk_audio = audio.VkAudio(vk_session)  # Получаем доступ к audio

os.chdir(path)
t = open('music.txt', 'w', encoding='Utf8')
n = open('not_found.txt', 'w', encoding='Utf8')

# Режим 1

if mode == '1':
    for i in vk_audio.get(owner_id=my_id):
        print(i["artist"] + ':' + i["title"], file=t)

# Режим 2
elif mode == '2':
  for i in vk_audio.get(owner_id=my_id):
    try:
        r = requests.get(i["url"])
        if r.status_code == REQUEST_STATUS_CODE:
            with open(i["artist"] + '_' + i["title"] + '.mp3', 'wb') as output_file:
                output_file.write(r.content)
    except OSError:
        print(i["artist"] + '_' + i["title"], file=n)
  t.close()

# Режим 3
else:
  for i in vk_audio.get(owner_id=my_id):
    try:
        r = requests.get(i["url"])
        if r.status_code == REQUEST_STATUS_CODE:
            with open(i["artist"] + '_' + i["title"] + '.mp3', 'wb') as output_file:
                output_file.write(r.content)
            print(i["artist"] + '_' + i["title"], file=t)
    except OSError:
        print(i["artist"] + '_' + i["title"], file=n)


Когда выпарсили всю музыку можем искать ее в Yandex.Music или же загружать их туда, если не найдем.

In [ ]:
def search_compare_add(vvod):
    artist, track_name = vvod.split(':')
    search_result = client.search(vvod)
    best_ = search_result.best
    try:
        type_ = best_.type
        best_result = best_.result
        if type_ == 'track':
            artist_searched = best_result.artists_name()
            track_name_searched = best_result.title
            ar = ''.join(artist_searched)
            if str(ar).lower() == str(artist).lower() and str(track_name).lower() == str(track_name_searched).lower():
                client.users_likes_tracks_add(best_result.id)
            elif str(track_name).lower() == str(track_name_searched).lower():
                answer = True
                for word in artist_searched:
                    if word.lower() not in str(artist).lower():
                        answer = False
                if answer:
                    client.users_likes_tracks_add(best_result.id)
                else:
                    print(vvod, file=not_found)
            else:
                print(vvod, file=not_found)
        else:
            print(vvod, file=not_found)
    except AttributeError:
        print(vvod, file=not_found)


In [ ]:
file = open('music.txt', 'r', encoding='utf8')
tracks = file.readlines()
not_found = open('Ненайдено', 'w', encoding='utf8')
email= input('Введите эл. почту:')
password = input('Введите пароль:')
client = Client.from_credentials('sokolov.super-kirill@yandex.ru', '30360120ftpore')
for track in tracks:
    r = track.strip()
    search_compare_add(r)
